In [2]:
import pyspark
from pyspark.sql import SparkSession 
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType, TimestampType
spark= SparkSession.builder.master('local[*]').appName('test').getOrCreate()

### Answers to hw 5 questions: 


##### q1: 3.3.2

##### q2: 24MB

##### q3: 452470

##### q4: 66.87

##### q5: 4040

##### q6:  Crown Heights North

### Qestion 2:

In [3]:
schema= StructType([
    StructField('dispatching_base_num', StringType(), True),
    StructField('pickup_datetime', TimestampType(), True),
    StructField('dropoff_datetime', TimestampType(), True),
    StructField('PULocationID', IntegerType(), True), 
    StructField('DOLocationID', IntegerType(), True), 
    StructField('SR_Flag', StringType(), True),
    StructField('Affiliated_base_number', StringType(), True)
    ])

df= spark.read.schema(schema).option('header', 'true').csv('fhvhv_tripdata_2021-06.csv.gz')
df.printSchema()


df= df.repartition(12)

df.write.mode('overwrite').parquet('fhvhv/2021/06')

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



23/03/04 17:54:28 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


### Qestion 3:

In [4]:
df_partitioned= spark.read.parquet('fhvhv/*/*') 

In [6]:

df_partitioned= df_partitioned.withColumn('pickup_date', to_date('pickup_datetime'))

df_partitioned.filter(df_partitioned.pickup_date=='2021-06-15').count()

452470

### question 4:

In [7]:
df_partitioned= df_partitioned.withColumn('trip_duration', \
                (unix_timestamp('dropoff_datetime')-unix_timestamp('pickup_datetime'))/3600)

# df.select(max(df.fee)).show()
df_partitioned.select(max(df_partitioned.trip_duration)).show()
# df_partitioned.filter(df_partitioned.trip_duration==df_partitioned.select(max(df_partitioned.trip_duration))).show()

+------------------+
|max(trip_duration)|
+------------------+
|  66.8788888888889|
+------------------+



In [8]:
df_partitioned.show(3)

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+-----------+-------------------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|Affiliated_base_number|pickup_date|      trip_duration|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+-----------+-------------------+
|              B02877|2021-06-11 12:28:02|2021-06-11 12:36:01|         230|          43|      N|                B02877| 2021-06-11|0.13305555555555557|
|              B02765|2021-06-13 08:46:22|2021-06-13 09:03:00|          37|          61|      N|                B02765| 2021-06-13| 0.2772222222222222|
|              B02510|2021-06-28 18:39:27|2021-06-28 18:50:53|         231|          79|      N|                  null| 2021-06-28|0.19055555555555556|
+--------------------+-------------------+-------------------+------------+------------+

### question 6:

In [9]:
!wget https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv

--2023-03-04 17:56:17--  https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.28.134, 52.217.39.206, 52.216.176.61, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.28.134|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12322 (12K) [application/octet-stream]
Saving to: ‘taxi+_zone_lookup.csv.1’

taxi+_zone_lookup.c 100%[===================>]  12.03K  --.-KB/s    in 0.001s  

2023-03-04 17:56:18 (22.5 MB/s) - ‘taxi+_zone_lookup.csv.1’ saved [12322/12322]



In [10]:
zones= spark.read.option('header', 'true').csv('taxi+_zone_lookup.csv')
zones.show(5)

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+
only showing top 5 rows



In [11]:
# col("department")
df_partitioned.join(zones, df_partitioned.PULocationID==zones.LocationID).\
    groupBy('Zone').count().sort(col("count").desc()).show(4)

+-------------------+------+
|               Zone| count|
+-------------------+------+
|Crown Heights North|231279|
|       East Village|221244|
|        JFK Airport|188867|
|     Bushwick South|187929|
+-------------------+------+
only showing top 4 rows

